In [ ]:
import pexpect
import os
import time
from getpass import getpass

def connect_via_jump(jump_host, jump_user, jump_pass, router_ip, router_user, router_pass):
    """Connect to router via jump host"""
    cmd = f'ssh -J {jump_user}@{jump_host} {router_user}@{router_ip}'
    child = pexpect.spawn(cmd)
    child.expect('password:')
    child.sendline(jump_pass)
    child.expect('password:')  # Router password prompt
    child.sendline(router_pass)
    return child

def collect_router_config(router_name, router_ip, jump_host, jump_user, jump_pass, router_user, router_pass, enable_pass):
    try:
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        output_dir = "router_configs"
        os.makedirs(output_dir, exist_ok=True)

        print(f"\nConnecting to {router_name} via jump host...")
        child = connect_via_jump(jump_host, jump_user, jump_pass, router_ip, router_user, router_pass)

        child.expect(['#', '>'], timeout=10)
        if child.after.decode() == '>':
            child.sendline('enable')
            child.expect('Password:')
            child.sendline(enable_pass)
            child.expect('#', timeout=5)

        child.sendline('terminal length 0')
        child.expect('#', timeout=5)

        # Get interfaces
        child.sendline('show ip interface brief')
        child.expect('#', timeout=10)
        with open(f"{output_dir}/{router_name}_interfaces.txt", 'w') as f:
            f.write(child.before.decode())

        # Get config
        child.sendline('show run')
        child.expect('#', timeout=30)
        with open(f"{output_dir}/{router_name}_config.txt", 'w') as f:
            f.write(child.before.decode())

        child.sendline('exit')
        return True

    except Exception as e:
        print(f"Error with {router_name}: {str(e)}")
        return False

def main():
    print("Router Config Collector via Jump Host")

    # Jump host details
    jump_host = input("Enter jump host IP: ")
    jump_user = input("Enter jump host username: ")
    jump_pass = getpass("Enter jump host password: ")

    # Router credentials
    router_user = input("Enter router username: ")
    router_pass = getpass("Enter router password: ")
    enable_pass = getpass("Enter enable password: ")

    # Router list (modify IPs as needed)
    routers = {
        'R1': '192.168.1.1',
        'R2': '192.168.1.2',
        'R3': '192.168.1.3',
        'R4': '192.168.1.4'
    }

    for name, ip in routers.items():
        if collect_router_config(name, ip, jump_host, jump_user, jump_pass,
                              router_user, router_pass, enable_pass):
            print(f"Success: {name}")
        else:
            print(f"Failed: {name}")

if __name__ == "__main__":
    main()